In [1]:
import pyodbc
import sys
import random
import pandas as pd


In [4]:
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=172.16.6.196,1433;"  # Use the IP address and port directly
    "Database=Demo1;"  
    "UID=wsl;" 
    "PWD=greatpond501;"  
)

print(conn_str)

conn = pyodbc.connect(conn_str)


Driver={ODBC Driver 17 for SQL Server};Server=172.16.6.196,1433;Database=Demo1;UID=wsl;PWD=greatpond501;
